# FASTQ Parser Benchmark Analysis

This notebook analyzes and visualizes hyperfine benchmark results comparing BBTools StreamerWrapper, fastqscan, needletail (Rust), and needletail (Python) FASTQ parsers.

## Cache Scenarios

- **Hot Cache**: Files copied to RAM disk (/tmp) for minimal I/O overhead - fastest scenario simulating in-memory access
- **Cold Cache**: Regular files with OS page cache cleared before each run - measures real-world first-access performance with full disk I/O
- **Really-Cold Cache**: Freshly generated files without cache clearing - measures parser performance on newly created data

In [2]:
%%bash
# pixi run bash ./run_all_benchmarks.sh --warmup 0 -r 1 --sizes "0.01m,0.1m,1m" 2>&1 | tee benchmark_run_$(date +%Y%m%d_%H%M%S).log # running externally
latest_log=$(ls -t benchmark_run_*.log 2>/dev/null | head -1)
cat "$latest_log"



Discovering available tools...
  Found: biofast-py1-4l
  Found: biofast-py2-rfq
  Found: biofast-py3-mappy
  Found: biofast-py4-bpitr
  Found: biofast-py5-bp
  Found: biofast-py6-pyfx
  Found: biofast-py7-pysam
  Found: fastqscan
  Found: fastqscanMT2c
  Found: needletail-python
  Found: needletail-rust
  Found: paraseq-filt
  Found: paraseq-filtMT2
  Found: paraseq-filtMT6
  Found: paraseq-filt_py
  Found: paraseq-filt_pyMT2c
  Found: polars-bio
Total tools available: 17
Using specified sizes: 0.1m,1m,20m,50m,70m,100m
Test files for size 20m not found. Generating...
Generating test files for size: 20m...
  Creating reference genome...
  Creating raw FASTQ (20000000 reads)...
java -ea --add-modules jdk.incubator.vector -Xmx59801m -cp /clusterfs/jgi/scratch/science/metagen/neri/code/blits/biofaster/BBTools/current/ synth.RandomReads3 build=1 ref=/clusterfs/jgi/scratch/science/metagen/neri/code/blits/biofaster/test-data/ref_genome_temp.fa out=/clusterfs/jgi/scratch/science/metagen/neri/

In [3]:
# Enable auto-reload of modules for development
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

# Add src directory to Python path
src_path = Path.cwd() / 'src'
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

import json
import altair as alt
import polars as pl
from biofaster.funcitons import load_hyperfine_results, plot_mean_times, find_latest_results_dir, parse_benchmark_data, compare_raw_vs_gzipped, plot_scatter_runs, plot_distributions, plot_really_cold_scaling, plot_test_size_scaling, plot_throughput

# Configure Altair for better display
alt.data_transformers.disable_max_rows()

# set printing of polars dataframes to show all columns and rows
pl.Config.set_fmt_str_lengths(100)
pl.Config.set_tbl_rows(100)
pl.Config.set_tbl_cols(50)

polars.config.Config

## Load Hyperfine JSON Results

Specify the paths to your benchmark result files. The script will look in `benchmark_results/` for the latest results or you can specify custom paths.

In [4]:
# Try to find latest results or specify custom paths
latest_dir = find_latest_results_dir()

if latest_dir:
    print(f"Found latest results in: {latest_dir}")
    
    # Results structure: organized by test size in subdirectories
    # e.g., benchmark_results/benchmark_TIMESTAMP/0.1m/hot_raw.json
    #                                             /1m/cold_gz.json
    #                                             /10m/really_cold_bgz.json
    
    result_files = {}
    
    # Discover all size subdirectories
    size_dirs = [d for d in latest_dir.iterdir() if d.is_dir() and not d.name.startswith('.')]
    
    if size_dirs:
        print(f"\nFound {len(size_dirs)} test size directories")
        
        # Load results from each size directory
        for size_dir in sorted(size_dirs):
            size_name = size_dir.name
            print(f"  Checking {size_name}/")
            
            # Define expected result files in each size directory
            # All cache types (hot, cold, really_cold) x all formats (raw, gz, bgz)
            size_results = {
                f'{size_name}_hot_raw': size_dir / 'hot_raw.json',
                f'{size_name}_hot_gz': size_dir / 'hot_gz.json',
                f'{size_name}_hot_bgz': size_dir / 'hot_bgz.json',
                f'{size_name}_cold_raw': size_dir / 'cold_raw.json',
                f'{size_name}_cold_gz': size_dir / 'cold_gz.json',
                f'{size_name}_cold_bgz': size_dir / 'cold_bgz.json',
                f'{size_name}_really_cold_raw': size_dir / 'really_cold_raw.json',
                f'{size_name}_really_cold_gz': size_dir / 'really_cold_gz.json',
                f'{size_name}_really_cold_bgz': size_dir / 'really_cold_bgz.json',
            }
            
            # Add files that exist
            for key, path in size_results.items():
                if path.exists():
                    result_files[key] = path
    
    # Filter to only existing files
    available_results = {k: v for k, v in result_files.items() if v.exists()}
    
    if available_results:
        print(f"\nTotal available results: {len(available_results)}")
        print("\nResults by category:")
        
        # Group by test size
        sizes = set()
        for key in available_results.keys():
            # Extract size from keys like '1m_hot_raw' or '1m_really_cold_raw'
            size = key.split('_')[0]
            sizes.add(size)
        
        for size in sorted(sizes):
            size_results = [k for k in available_results.keys() if k.startswith(f'{size}_')]
            if size_results:
                print(f"  {size}: {len(size_results)} benchmarks")
    else:
        print("\n⚠️ No benchmark result files found")
        
else:
    print("No benchmark results found. Run ./run_all_benchmarks.sh first.")
    #   manually specify paths here:
    available_results = {}

Found latest results in: benchmark_results/benchmark_20251215_115019

Found 6 test size directories
  Checking 0.1m/
  Checking 100m/
  Checking 1m/
  Checking 20m/
  Checking 50m/
  Checking 70m/

Total available results: 36

Results by category:
  0.1m: 6 benchmarks
  100m: 6 benchmarks
  1m: 6 benchmarks
  20m: 6 benchmarks
  50m: 6 benchmarks
  70m: 6 benchmarks


## System Information

Display the system configuration where benchmarks were run.

In [5]:
# Display system information
system_info_path = latest_dir / 'system_info.json' if latest_dir else None

with open(system_info_path) as f:
    system_info = json.load(f)

print("=" * 60)
print("BENCHMARK SYSTEM INFORMATION")
print("=" * 60)
for key, value in system_info.items():
        print(f"{key}: {value}")


BENCHMARK SYSTEM INFORMATION
os: GNU/Linux
kernel: 4.18.0-553.58.1.el8_10.x86_64
architecture: x86_64
cpu: AMD EPYC 7543 32-Core Processor
cpu_cores: 8
cpu_threads: 64
cpu_frequency: 2794.578 MHz
cpu_cache_l3: 32768K
ram: 503Gi
ram_total_bytes: 540619436032
disk_device: vast1.jgi:/jgi_scratch
filesystem: nfs
disk_total: 1.3P
disk_available: 204T
disk_used_percent: 85%
disk_model: HFS3T8G3H2X069N
python_version: 3.12.12
python_path: /clusterfs/jgi/scratch/science/metagen/neri/code/blits/biofaster/.pixi/envs/default/bin/python3
hyperfine_version: 1.20.0
java_version: 25.0.1
cache_drop_available: false
timestamp: 2025-12-15T19:53:22Z
date_local: Mon Dec 15 11:53:22 PST 2025


## Parse Benchmark Data

Extract metrics from hyperfine JSON including mean times, standard deviations, min/max, and individual runs.

**Note:** Benchmarks are run with `--ignore-failure` flag, so tools that fail (e.g., Python parsers that can't handle gzipped files) will be skipped and not appear in results. Failed commands will show warnings during data loading.

In [6]:
available_results

{'0.1m_hot_raw': PosixPath('benchmark_results/benchmark_20251215_115019/0.1m/hot_raw.json'),
 '0.1m_hot_gz': PosixPath('benchmark_results/benchmark_20251215_115019/0.1m/hot_gz.json'),
 '0.1m_hot_bgz': PosixPath('benchmark_results/benchmark_20251215_115019/0.1m/hot_bgz.json'),
 '0.1m_cold_raw': PosixPath('benchmark_results/benchmark_20251215_115019/0.1m/cold_raw.json'),
 '0.1m_cold_gz': PosixPath('benchmark_results/benchmark_20251215_115019/0.1m/cold_gz.json'),
 '0.1m_cold_bgz': PosixPath('benchmark_results/benchmark_20251215_115019/0.1m/cold_bgz.json'),
 '100m_hot_raw': PosixPath('benchmark_results/benchmark_20251215_115019/100m/hot_raw.json'),
 '100m_hot_gz': PosixPath('benchmark_results/benchmark_20251215_115019/100m/hot_gz.json'),
 '100m_hot_bgz': PosixPath('benchmark_results/benchmark_20251215_115019/100m/hot_bgz.json'),
 '100m_cold_raw': PosixPath('benchmark_results/benchmark_20251215_115019/100m/cold_raw.json'),
 '100m_cold_gz': PosixPath('benchmark_results/benchmark_20251215_115

In [7]:
# Load all available results
# Note: With --ignore-failure flag, failed commands will be skipped
# and warnings will be printed for any tools that didn't complete successfully
all_data = {}
for name, path in available_results.items():
    print(f"Loading {name}...")
    df = parse_benchmark_data(path)
    all_data[name] = df
    if len(df) > 0:
        print(df[['command', 'mean', 'stddev', 'min', 'max']])
    print()

Loading 0.1m_hot_raw...
⚠️  Command 'polars-bio' failed 4/4 runs - excluding from results
shape: (16, 5)
┌─────────────────────┬──────────┬──────────┬──────────┬──────────┐
│ command             ┆ mean     ┆ stddev   ┆ min      ┆ max      │
│ ---                 ┆ ---      ┆ ---      ┆ ---      ┆ ---      │
│ str                 ┆ f64      ┆ f64      ┆ f64      ┆ f64      │
╞═════════════════════╪══════════╪══════════╪══════════╪══════════╡
│ biofast-py2-rfq     ┆ 0.298456 ┆ 0.012312 ┆ 0.283214 ┆ 0.317724 │
│ paraseq-filt_pyMT2c ┆ 0.107968 ┆ 0.004427 ┆ 0.10189  ┆ 0.118486 │
│ paraseq-filtMT2     ┆ 0.024865 ┆ 0.001254 ┆ 0.023447 ┆ 0.030506 │
│ needletail-rust     ┆ 0.022014 ┆ 0.000957 ┆ 0.020576 ┆ 0.026271 │
│ paraseq-filtMT6     ┆ 0.027755 ┆ 0.001134 ┆ 0.026572 ┆ 0.035964 │
│ biofast-py3-mappy   ┆ 0.300691 ┆ 0.007982 ┆ 0.288531 ┆ 0.313099 │
│ biofast-py7-pysam   ┆ 0.456364 ┆ 0.007282 ┆ 0.447513 ┆ 0.464813 │
│ biofast-py1-4l      ┆ 0.252522 ┆ 0.009303 ┆ 0.242475 ┆ 0.273    │
│ biofast-p

## Failed Commands Report

Commands with non-zero exit codes are automatically excluded from all plots and analysis.

In [8]:
# Check for failed commands in the JSON results
print("Checking for commands with failures (non-zero exit codes)...")
print("=" * 70)

failed_commands_found = False

for name, path in available_results.items():
    # Load the raw JSON to check exit codes
    with open(path, 'r') as f:
        data = json.load(f)
    
    failed_in_this_benchmark = []
    
    for result in data['results']:
        print(result)
        command = result.get('command', 'unknown')
        exit_codes = result.get('exit_codes', [])
        
        if exit_codes and any(code != 0 for code in exit_codes):
            failed_runs = sum(1 for code in exit_codes if code != 0)
            total_runs = len(exit_codes)
            failed_in_this_benchmark.append({
                'command': command,
                'failed_runs': failed_runs,
                'total_runs': total_runs,
                'exit_codes': exit_codes
            })
    
    if failed_in_this_benchmark:
        failed_commands_found = True
        print(f"\n{name}:")
        for item in failed_in_this_benchmark:
            print(f"  ❌ {item['command']}")
            print(f"     Failed: {item['failed_runs']}/{item['total_runs']} runs")
            print(f"     Exit codes: {item['exit_codes']}")

if not failed_commands_found:
    print("\n✅ No failed commands detected - all tools completed successfully!")
    
print("\n" + "=" * 70)

Checking for commands with failures (non-zero exit codes)...
{'command': 'biofast-py2-rfq', 'mean': 0.2984556315044445, 'stddev': 0.01231185731951459, 'median': 0.29595823306, 'user': 0.2226297222222222, 'system': 0.0517028911111111, 'min': 0.28321364606000005, 'max': 0.31772385806000003, 'times': [0.29871689406, 0.31772385806000003, 0.28943015606, 0.28715815306000003, 0.29595823306, 0.31366659806, 0.28321364606000005, 0.29126266006, 0.30897048506], 'memory_usage_byte': [89952256, 89952256, 89952256, 89952256, 89952256, 89952256, 89952256, 89952256, 89952256], 'exit_codes': [0, 0, 0, 0, 0, 0, 0, 0, 0]}
{'command': 'paraseq-filt_pyMT2c', 'mean': 0.10796791817538462, 'stddev': 0.004427068770312667, 'median': 0.10703807006000002, 'user': 0.045661653846153845, 'system': 0.050887626153846156, 'min': 0.10189049006, 'max': 0.11848601706000002, 'times': [0.10773218306000001, 0.10571446906000001, 0.10827076906000001, 0.10918991206000002, 0.10682973206000002, 0.10753453006000001, 0.1184841240600

In [9]:
# Extract file sizes for display in charts
from biofaster.funcitons import get_file_size_from_results

file_sizes = {}
for name, path in available_results.items():
    size = get_file_size_from_results(path)
    if size:
        file_sizes[name] = size
        print(f"{name}: {size}")

print(f"\nFile sizes extracted for {len(file_sizes)} benchmarks")

0.1m_hot_raw: 67.3 MB, raw
0.1m_hot_gz: 10.6 MB, gzip
0.1m_hot_bgz: 11.2 MB, bgzip
0.1m_cold_raw: 67.3 MB, raw
0.1m_cold_gz: 10.6 MB, gzip
0.1m_cold_bgz: 11.2 MB, bgzip
100m_hot_raw: 33.04 GB, raw
100m_hot_gz: 15.12 GB, gzip
100m_hot_bgz: 15.05 GB, bgzip
100m_cold_raw: 33.04 GB, raw
100m_cold_gz: 15.12 GB, gzip
100m_cold_bgz: 15.05 GB, bgzip
1m_hot_raw: 673.1 MB, raw
1m_hot_gz: 105.6 MB, gzip
1m_hot_bgz: 112.2 MB, bgzip
1m_cold_raw: 673.1 MB, raw
1m_cold_gz: 105.6 MB, gzip
1m_cold_bgz: 112.2 MB, bgzip
20m_hot_raw: 6.60 GB, raw
20m_hot_gz: 3.02 GB, gzip
20m_hot_bgz: 2.20 GB, bgzip
20m_cold_raw: 6.60 GB, raw
20m_cold_gz: 3.02 GB, gzip
20m_cold_bgz: 2.20 GB, bgzip
50m_hot_raw: 32.87 GB, raw
50m_hot_gz: 5.16 GB, gzip
50m_hot_bgz: 5.48 GB, bgzip
50m_cold_raw: 32.87 GB, raw
50m_cold_gz: 5.16 GB, gzip
50m_cold_bgz: 5.48 GB, bgzip
70m_hot_raw: 46.01 GB, raw
70m_hot_gz: 7.22 GB, gzip
70m_hot_bgz: 7.67 GB, bgzip
70m_cold_raw: 46.01 GB, raw
70m_cold_gz: 7.22 GB, gzip
70m_cold_bgz: 7.67 GB, bgzip


## Bar Plot: Mean Execution Times

Compare mean execution times between all parsers with error bars showing standard deviation.

In [10]:
# Plot all available results
chart = plot_mean_times(all_data, "FASTQ Parser Benchmark: Mean Execution Times", file_sizes=file_sizes)
chart

/tmp/ipykernel_2326176/1651240058.py:2: UserWarning: Automatically deduplicated selection parameter with identical configuration. If you want independent parameters, explicitly name them differently (e.g., name='param1', name='param2'). See https://github.com/vega/altair/issues/3891
  chart = plot_mean_times(all_data, "FASTQ Parser Benchmark: Mean Execution Times", file_sizes=file_sizes)


alt.VConcatChart(...)

In [11]:
# Debug: Check how many commands in each benchmark
for name, df in sorted(all_data.items()):
    if 'hot' in name and 'raw' in name:
        print(f"{name}: {len(df)} commands")

0.1m_hot_raw: 16 commands
100m_hot_raw: 16 commands
1m_hot_raw: 16 commands
20m_hot_raw: 16 commands
50m_hot_raw: 16 commands
70m_hot_raw: 16 commands


## Distribution Plots: Individual Run Times

Visualize the distribution of execution times across all runs using box plots.

In [12]:
# Plot distributions
chart = plot_distributions(all_data, file_sizes=file_sizes)
chart

/tmp/ipykernel_2326176/907112968.py:2: UserWarning: Automatically deduplicated selection parameter with identical configuration. If you want independent parameters, explicitly name them differently (e.g., name='param1', name='param2'). See https://github.com/vega/altair/issues/3891
  chart = plot_distributions(all_data, file_sizes=file_sizes)


alt.VConcatChart(...)

## Side-by-Side Comparison: Raw vs Gzipped

Compare performance between raw and gzipped FASTQ files for each parser.

In [13]:
chart = compare_raw_vs_gzipped(all_data)
if chart:
    display(chart)
else:
    print("Not enough data available for raw vs gzipped comparison")



=== Performance Analysis ===

--- Size: 0.1m ---

biofast-py1-4l:
  Raw:     0.25s
  Gzipped: 0.45s
  Ratio:   1.79x slower

biofast-py2-rfq:
  Raw:     0.30s
  Gzipped: 0.52s
  Ratio:   1.74x slower

biofast-py3-mappy:
  Raw:     0.30s
  Gzipped: 0.41s
  Ratio:   1.37x slower

biofast-py4-bpitr:
  Raw:     0.84s
  Gzipped: 1.00s
  Ratio:   1.19x slower

biofast-py5-bp:
  Raw:     1.52s
  Gzipped: 1.67s
  Ratio:   1.10x slower

biofast-py6-pyfx:
  Raw:     0.24s
  Gzipped: 0.35s
  Ratio:   1.44x slower

biofast-py7-pysam:
  Raw:     0.46s
  Gzipped: 0.57s
  Ratio:   1.24x slower

fastqscan:
  Raw:     0.21s
  Gzipped: 0.28s
  Ratio:   1.33x slower

fastqscanMT2c:
  Raw:     0.21s
  Gzipped: 0.28s
  Ratio:   1.36x slower

needletail-python:
  Raw:     0.19s
  Gzipped: 0.27s
  Ratio:   1.46x slower

needletail-rust:
  Raw:     0.02s
  Gzipped: 0.10s
  Ratio:   4.42x slower

paraseq-filt:
  Raw:     0.03s
  Gzipped: 0.10s
  Ratio:   3.81x slower

paraseq-filtMT2:
  Raw:     0.02s
  Gzipp

alt.LayerChart(...)

## Scatter Plot: Individual Run Times

Visualize individual run times to identify outliers and performance consistency.

## Compression Comparison: Gzip vs Bgzip

Compare performance between gzip and bgzip compression formats to evaluate decompression overhead.

## Test Size Scaling Analysis

Analyze how each parser scales with file size across different test datasets (0.1M, 0.5M, 1M, 10M, 50M reads).

In [14]:
# Plot scaling for hot cache, raw files
chart = plot_test_size_scaling(all_data, cache_type='hot', file_format='raw')
if chart:
    display(chart)
else:
    print("Not enough data for hot cache raw scaling analysis")


=== Hot Cache Raw FASTQ Scaling Analysis ===

biofast-py1-4l:
  0.1M  :   0.253s ±  0.009s
  1M    :   1.870s ±  0.112s
  20M   :  17.442s ±  0.119s
  50M   :  92.098s ±  1.265s
  70M   : 121.917s ±  1.091s
  100M  :  87.834s ±  1.541s
  Scaling: 347.83x time for 1000.0x data
  Efficiency: 0.35 (1.0 = linear)

biofast-py2-rfq:
  0.1M  :   0.298s ±  0.012s
  1M    :   2.213s ±  0.031s
  20M   :  21.945s ±  0.121s
  50M   : 110.859s ±  3.407s
  70M   : 161.120s ± 13.070s
  100M  : 113.739s ±  8.292s
  Scaling: 381.09x time for 1000.0x data
  Efficiency: 0.38 (1.0 = linear)

biofast-py3-mappy:
  0.1M  :   0.301s ±  0.008s
  1M    :   1.631s ±  0.005s
  20M   :  15.181s ±  0.029s
  50M   :  76.463s ±  1.898s
  70M   : 105.780s ±  3.356s
  100M  :  75.106s ±  1.127s
  Scaling: 249.78x time for 1000.0x data
  Efficiency: 0.25 (1.0 = linear)

biofast-py4-bpitr:
  0.1M  :   0.841s ±  0.028s
  1M    :   2.681s ±  0.090s
  20M   :  21.583s ±  0.288s
  50M   : 106.038s ±  7.750s
  70M   : 163.69

alt.LayerChart(...)

In [15]:
# Plot scaling for hot cache, gzipped files
chart = plot_test_size_scaling(all_data, cache_type='hot', file_format='gz')
if chart:
    display(chart)
else:
    print("Not enough data for hot cache gzipped scaling analysis")


=== Hot Cache Gzipped Scaling Analysis ===

biofast-py1-4l:
  0.1M  :   0.452s ±  0.013s
  1M    :   3.685s ±  0.034s
  20M   :  54.872s ±  0.062s
  50M   : 181.653s ±  1.099s
  70M   : 255.625s ±  5.138s
  100M  : 272.616s ±  0.222s
  Scaling: 603.28x time for 1000.0x data
  Efficiency: 0.60 (1.0 = linear)

biofast-py2-rfq:
  0.1M  :   0.518s ±  0.011s
  1M    :   4.390s ±  0.075s
  20M   :  62.938s ±  0.487s
  50M   : 218.756s ±  4.735s
  70M   : 307.446s ±  3.650s
  100M  : 309.214s ±  1.859s
  Scaling: 596.93x time for 1000.0x data
  Efficiency: 0.60 (1.0 = linear)

biofast-py3-mappy:
  0.1M  :   0.412s ±  0.010s
  1M    :   2.778s ±  0.023s
  20M   :  44.275s ±  0.137s
  50M   : 130.810s ±  0.960s
  70M   : 181.098s ±  0.371s
  100M  : 222.042s ±  1.827s
  Scaling: 538.70x time for 1000.0x data
  Efficiency: 0.54 (1.0 = linear)

biofast-py4-bpitr:
  0.1M  :   1.004s ±  0.021s
  1M    :   4.764s ±  0.034s
  20M   :  60.291s ±  0.272s
  50M   : 216.096s ±  2.965s
  70M   : 290.245s

alt.LayerChart(...)

## Throughput Analysis

Calculate and visualize throughput (GB/s) based on actual file sizes and mean execution times.

In [16]:
# Plot throughput for hot cache, raw files
chart = plot_throughput(all_data, available_results, cache_type='hot', file_format='raw')
if chart:
    display(chart)
else:
    print("Not enough data for hot cache raw throughput analysis")


=== Hot Cache Raw FASTQ Throughput Analysis ===

biofast-py1-4l:
  0.1M  :   0.260 GB/s ± 0.0096 (67.3 MB)
  1M    :   0.352 GB/s ± 0.0211 (673.1 MB)
  20M   :   0.378 GB/s ± 0.0026 (6757.4 MB)
  50M   :   0.357 GB/s ± 0.0049 (33654.1 MB)
  70M   :   0.377 GB/s ± 0.0034 (47115.7 MB)
  100M  :   0.376 GB/s ± 0.0066 (33829.6 MB)
  Average: 0.350 GB/s

biofast-py2-rfq:
  0.1M  :   0.220 GB/s ± 0.0091 (67.3 MB)
  1M    :   0.297 GB/s ± 0.0042 (673.1 MB)
  20M   :   0.301 GB/s ± 0.0017 (6757.4 MB)
  50M   :   0.296 GB/s ± 0.0091 (33654.1 MB)
  70M   :   0.286 GB/s ± 0.0232 (47115.7 MB)
  100M  :   0.290 GB/s ± 0.0212 (33829.6 MB)
  Average: 0.282 GB/s

biofast-py3-mappy:
  0.1M  :   0.219 GB/s ± 0.0058 (67.3 MB)
  1M    :   0.403 GB/s ± 0.0011 (673.1 MB)
  20M   :   0.435 GB/s ± 0.0008 (6757.4 MB)
  50M   :   0.430 GB/s ± 0.0107 (33654.1 MB)
  70M   :   0.435 GB/s ± 0.0138 (47115.7 MB)
  100M  :   0.440 GB/s ± 0.0066 (33829.6 MB)
  Average: 0.393 GB/s

biofast-py4-bpitr:
  0.1M  :   0.078 

alt.LayerChart(...)

In [17]:
# Plot throughput for hot cache, gzipped files
chart = plot_throughput(all_data, available_results, cache_type='hot', file_format='gz')
if chart:
    display(chart)
else:
    print("Not enough data for hot cache gzipped throughput analysis")


=== Hot Cache Gzipped Throughput Analysis ===

biofast-py1-4l:
  0.1M  :   0.023 GB/s ± 0.0006 (10.6 MB)
  1M    :   0.028 GB/s ± 0.0003 (105.6 MB)
  20M   :   0.055 GB/s ± 0.0001 (3096.0 MB)
  50M   :   0.028 GB/s ± 0.0002 (5283.1 MB)
  70M   :   0.028 GB/s ± 0.0006 (7396.3 MB)
  100M  :   0.055 GB/s ± 0.0000 (15479.5 MB)
  Average: 0.036 GB/s

biofast-py2-rfq:
  0.1M  :   0.020 GB/s ± 0.0004 (10.6 MB)
  1M    :   0.024 GB/s ± 0.0004 (105.6 MB)
  20M   :   0.048 GB/s ± 0.0004 (3096.0 MB)
  50M   :   0.024 GB/s ± 0.0005 (5283.1 MB)
  70M   :   0.023 GB/s ± 0.0003 (7396.3 MB)
  100M  :   0.049 GB/s ± 0.0003 (15479.5 MB)
  Average: 0.031 GB/s

biofast-py3-mappy:
  0.1M  :   0.025 GB/s ± 0.0006 (10.6 MB)
  1M    :   0.037 GB/s ± 0.0003 (105.6 MB)
  20M   :   0.068 GB/s ± 0.0002 (3096.0 MB)
  50M   :   0.039 GB/s ± 0.0003 (5283.1 MB)
  70M   :   0.040 GB/s ± 0.0001 (7396.3 MB)
  100M  :   0.068 GB/s ± 0.0006 (15479.5 MB)
  Average: 0.046 GB/s

biofast-py4-bpitr:
  0.1M  :   0.010 GB/s ± 0

alt.LayerChart(...)

In [18]:
# Compare gzip vs bgzip if both are available
# Now handles size-prefixed keys (e.g., 0.1m_hot_gz, 0.5m_hot_bgz)

# Find all gz and bgz results with size prefixes
gz_keys = [k for k in all_data.keys() if '_gz' in k and '_bgz' not in k and len(all_data[k]) > 0]
bgz_keys = [k for k in all_data.keys() if '_bgz' in k and len(all_data[k]) > 0]

if not gz_keys or not bgz_keys:
    print("Gzip or Bgzip results not found. Run benchmarks with both compression formats.")
    print("Make sure bgzip is installed and run: ./run_all_benchmarks.sh")
else:
    # Group by cache type and size
    comparisons = {}
    
    for gz_key in gz_keys:
        # Extract size and cache type from key (e.g., "0.1m_hot_gz" -> "0.1m", "hot")
        parts = gz_key.split('_')
        if len(parts) >= 3:
            size = parts[0]
            cache_type = parts[1]  # hot or cold
            
            # Find corresponding bgz key
            bgz_key = f"{size}_{cache_type}_bgz"
            
            if bgz_key in all_data and len(all_data[bgz_key]) > 0:
                comp_key = f"{size}_{cache_type}"
                comparisons[comp_key] = {
                    'size': size,
                    'cache_type': cache_type,
                    'gz': all_data[gz_key],
                    'bgz': all_data[bgz_key]
                }
    
    if comparisons:
        # Create charts for each comparison
        all_charts = []
        
        for comp_key, comp_data in sorted(comparisons.items()):
            size = comp_data['size']
            cache_type = comp_data['cache_type']
            
            # Combine data
            df_gz = comp_data['gz'].with_columns(pl.lit('gzip').alias('compression'))
            df_bgz = comp_data['bgz'].with_columns(pl.lit('bgzip').alias('compression'))
            combined = pl.concat([df_gz, df_bgz])
            
            # Create grouped bar chart
            chart = alt.Chart(combined).mark_bar(opacity=0.8).encode(
                x=alt.X('command:N', title='Parser', axis=alt.Axis(labelAngle=-45)),
                y=alt.Y('mean:Q', title='Time (seconds)'),
                color=alt.Color('compression:N',
                              scale=alt.Scale(domain=['gzip', 'bgzip'],
                                            range=['#3498db', '#e74c3c']),
                              legend=alt.Legend(title='Compression')),
                xOffset='compression:N',
                tooltip=[
                    alt.Tooltip('command:N', title='Parser'),
                    alt.Tooltip('compression:N', title='Type'),
                    alt.Tooltip('mean:Q', title='Mean', format='.3f'),
                    alt.Tooltip('stddev:Q', title='Std Dev', format='.3f')
                ]
            ).properties(
                title=f'{size.upper()} {cache_type.title()} Cache: Gzip vs Bgzip',
                width=400,
                height=400
            )
            
            # Add error bars
            error_bars = alt.Chart(combined).mark_errorbar(extent='stdev').encode(
                x=alt.X('command:N'),
                y=alt.Y('mean:Q'),
                yError='stddev:Q',
                xOffset='compression:N'
            )
            
            all_charts.append(chart + error_bars)
        
        # Combine charts in a grid layout (2 per row)
        if len(all_charts) == 1:
            final_chart = all_charts[0]
        elif len(all_charts) == 2:
            final_chart = alt.hconcat(*all_charts)
        else:
            # Create rows of 2 charts each
            rows = []
            for i in range(0, len(all_charts), 2):
                if i + 1 < len(all_charts):
                    rows.append(alt.hconcat(all_charts[i], all_charts[i + 1]))
                else:
                    rows.append(all_charts[i])
            final_chart = alt.vconcat(*rows)
        
        final_chart = final_chart.configure_axis(gridOpacity=0.3)
        
        # Save chart
        final_chart.save(str("plots/compression_comparison.html"))
        
        # Display
        display(final_chart)
        
        # Print speedup ratios
        print("\nCompression Speedup Analysis (gzip time / bgzip time):")
        print("=" * 70)
        
        for comp_key, comp_data in sorted(comparisons.items()):
            size = comp_data['size']
            cache_type = comp_data['cache_type']
            df_gz = comp_data['gz']
            df_bgz = comp_data['bgz']
            
            print(f"\n{size.upper()} {cache_type.title()} Cache:")
            
            commands = sorted(set(df_gz['command'].to_list()))
            for cmd in commands:
                gz_rows = df_gz.filter(pl.col('command') == cmd)
                bgz_rows = df_bgz.filter(pl.col('command') == cmd)
                
                if len(gz_rows) > 0 and len(bgz_rows) > 0:
                    gz_mean = gz_rows['mean'][0]
                    bgz_mean = bgz_rows['mean'][0]
                    speedup = gz_mean / bgz_mean
                    
                    if speedup > 1:
                        print(f"  {cmd}: {speedup:.2f}x faster with bgzip")
                    elif speedup < 1:
                        print(f"  {cmd}: {1/speedup:.2f}x faster with gzip")
                    else:
                        print(f"  {cmd}: Equal performance")
    else:
        print("No matching gzip/bgzip pairs found for comparison.")

alt.VConcatChart(...)


Compression Speedup Analysis (gzip time / bgzip time):

0.1M Cold Cache:
  biofast-py1-4l: 1.07x faster with gzip
  biofast-py2-rfq: 1.08x faster with gzip
  biofast-py3-mappy: 1.02x faster with gzip
  biofast-py4-bpitr: 1.02x faster with gzip
  biofast-py5-bp: 1.02x faster with gzip
  biofast-py6-pyfx: 1.04x faster with gzip
  biofast-py7-pysam: 1.03x faster with gzip
  fastqscan: 1.23x faster with bgzip
  fastqscanMT2c: 1.17x faster with bgzip
  needletail-python: 1.03x faster with gzip
  needletail-rust: 1.03x faster with gzip
  paraseq-filt: 1.03x faster with gzip
  paraseq-filtMT2: 1.02x faster with gzip
  paraseq-filtMT6: 1.01x faster with gzip
  paraseq-filt_py: 1.02x faster with gzip
  paraseq-filt_pyMT2c: 1.01x faster with gzip

0.1M Hot Cache:
  biofast-py1-4l: 1.07x faster with gzip
  biofast-py2-rfq: 1.08x faster with gzip
  biofast-py3-mappy: 1.00x faster with gzip
  biofast-py4-bpitr: 1.05x faster with gzip
  biofast-py5-bp: 1.07x faster with gzip
  biofast-py6-pyfx: 1.0

## Summary Statistics Table

Display a comprehensive table with all benchmark statistics.

## Really-Cold Cache Analysis: Generated Files

Analyze performance on freshly generated FASTQ files of various sizes to measure true cold-start performance including file generation overhead.

In [23]:
chart = plot_really_cold_scaling(all_data)
chart

No really-cold cache results found.


Finally summarize key findings and insights from the benchmark analysis.

In [20]:
summary_rows = []

for name, df in all_data.items():
    for row in df.iter_rows(named=True):
        summary_rows.append({
            'Benchmark': name.replace('_', ' ').title(),
            'Parser': row['command'],
            'Mean (s)': f"{row['mean']:.3f}",
            'Std Dev (s)': f"{row['stddev']:.3f}",
            'Median (s)': f"{row['median']:.3f}",
            'Min (s)': f"{row['min']:.3f}",
            'Max (s)': f"{row['max']:.3f}",
            'Runs': len(row['times'])
        })

summary_df = pl.DataFrame(summary_rows)

summary_df.write_csv(latest_dir / 'benchmark_summary.csv')
print("Benchmark summary:")
print(summary_df)

Benchmark summary:
shape: (576, 8)
┌───────────────┬─────────────────┬──────────┬─────────────┬────────────┬─────────┬─────────┬──────┐
│ Benchmark     ┆ Parser          ┆ Mean (s) ┆ Std Dev (s) ┆ Median (s) ┆ Min (s) ┆ Max (s) ┆ Runs │
│ ---           ┆ ---             ┆ ---      ┆ ---         ┆ ---        ┆ ---     ┆ ---     ┆ ---  │
│ str           ┆ str             ┆ str      ┆ str         ┆ str        ┆ str     ┆ str     ┆ i64  │
╞═══════════════╪═════════════════╪══════════╪═════════════╪════════════╪═════════╪═════════╪══════╡
│ 0.1M Hot Raw  ┆ biofast-py2-rfq ┆ 0.298    ┆ 0.012       ┆ 0.296      ┆ 0.283   ┆ 0.318   ┆ 9    │
│ 0.1M Hot Raw  ┆ paraseq-filt_py ┆ 0.108    ┆ 0.004       ┆ 0.107      ┆ 0.102   ┆ 0.118   ┆ 26   │
│               ┆ MT2c            ┆          ┆             ┆            ┆         ┆         ┆      │
│ 0.1M Hot Raw  ┆ paraseq-filtMT2 ┆ 0.025    ┆ 0.001       ┆ 0.025      ┆ 0.023   ┆ 0.031   ┆ 96   │
│ 0.1M Hot Raw  ┆ needletail-rust ┆ 0.022    ┆ 0.001    

In [21]:
# Copy summary files to plots directory for GitHub Pages
import shutil

plots_dir = Path('plots')
plots_dir.mkdir(exist_ok=True)

# Copy CSV summary (generated from all results)
if (latest_dir / 'benchmark_summary.csv').exists():
    shutil.copy(latest_dir / 'benchmark_summary.csv', plots_dir / 'benchmark_summary.csv')
    print("✓ Copied benchmark_summary.csv to plots/")

# Copy SUMMARY.txt
if (latest_dir / 'SUMMARY.txt').exists():
    shutil.copy(latest_dir / 'SUMMARY.txt', plots_dir / 'SUMMARY.txt')
    print("✓ Copied SUMMARY.txt to plots/")

# Copy system info
if (latest_dir / 'system_info.json').exists():
    shutil.copy(latest_dir / 'system_info.json', plots_dir / 'system_info.json')
    print("✓ Copied system_info.json to plots/")
    sys_info = dict(json.load(open(latest_dir / 'system_info.json')))
    # Print benchmark system info in a readable format
    print("\nBenchmark system info:")
    print(json.dumps(sys_info, indent=2))

# Copy markdown files from all size subdirectories
md_count = 0
for size_dir in latest_dir.iterdir():
    if size_dir.is_dir() and not size_dir.name.startswith('.'):
        size_name = size_dir.name
        for md_file in size_dir.glob('*.md'):
            # Add size prefix to avoid name collisions
            dest_name = f"{size_name}_{md_file.name}"
            shutil.copy(md_file, plots_dir / dest_name)
            md_count += 1

# Also copy root-level markdown files (really-cold benchmarks)
for md_file in latest_dir.glob('*.md'):
    if md_file.is_file():
        shutil.copy(md_file, plots_dir / md_file.name)
        md_count += 1

if md_count > 0:
    print(f"✓ Copied {md_count} markdown files to plots/")

print(f"\n✅ All files ready for GitHub Pages in plots/ directory")


✓ Copied benchmark_summary.csv to plots/
✓ Copied SUMMARY.txt to plots/
✓ Copied system_info.json to plots/

Benchmark system info:
{
  "os": "GNU/Linux",
  "kernel": "4.18.0-553.58.1.el8_10.x86_64",
  "architecture": "x86_64",
  "cpu": "AMD EPYC 7543 32-Core Processor",
  "cpu_cores": "8",
  "cpu_threads": "64",
  "cpu_frequency": "2794.578 MHz",
  "cpu_cache_l3": "32768K",
  "ram": "503Gi",
  "ram_total_bytes": "540619436032",
  "disk_device": "vast1.jgi:/jgi_scratch",
  "filesystem": "nfs",
  "disk_total": "1.3P",
  "disk_available": "204T",
  "disk_used_percent": "85%",
  "disk_model": "HFS3T8G3H2X069N",
  "python_version": "3.12.12",
  "python_path": "/clusterfs/jgi/scratch/science/metagen/neri/code/blits/biofaster/.pixi/envs/default/bin/python3",
  "hyperfine_version": "1.20.0",
  "java_version": "25.0.1",
  "cache_drop_available": "false",
  "timestamp": "2025-12-15T19:53:22Z",
  "date_local": "Mon Dec 15 11:53:22 PST 2025"
}
✓ Copied 36 markdown files to plots/

✅ All files rea

## Generate GitHub Pages Index

Generate the main index.html file with embedded charts for GitHub Pages deployment.

In [22]:
# Generate index.html for GitHub Pages
from biofaster.generate_index import generate_index_html

print("Generating index.html...")
generate_index_html()
print("✅ index.html generated successfully in plots/ directory")

Generating index.html...
✅ Generated plots/index.html
✅ index.html generated successfully in plots/ directory
